Part I: EDA
Do EDA task:
1. Df payment_enriched (Merge payment_report.csv with product.csv)
2. Df transactions

Suggestions:
1. Check each column: missing data? duplicates? incorrect data types?
2. Summarize numerical data: any incorrect values?

Sample Answers:
- Incorrect data types: column A, column B -> Next step: No action/ Delete rows/…
- Incorrect values: column A, column B -> Next step: No action/ Delete rows/…
- Missing data: x rows in column A, y rows in column B -> Next step: No action/ Delete rows/…
- Duplicates: PK? x rows? -> Next step: No action/ Delete rows/…



In [ ]:
import pandas as pd
import numpy as np

# define payment_enriched, transactions
transactions = pd.read_csv('transactions.csv')
payment_report = pd.read_csv('payment_report.csv')
product = pd.read_csv('product.csv')

payment_enriched = pd.merge(payment_report, product, on="product_id")

In [ ]:
# Checking Data types
payment_enriched.info()
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897 entries, 0 to 896
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   report_month   897 non-null    object
 1   payment_group  897 non-null    object
 2   product_id     897 non-null    int64 
 3   source_id      897 non-null    int64 
 4   volume         897 non-null    int64 
 5   category       897 non-null    object
 6   team_own       897 non-null    object
dtypes: int64(3), object(4)
memory usage: 49.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324002 entries, 0 to 1324001
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   transaction_id  1324002 non-null  int64  
 1   merchant_id     1324002 non-null  int64  
 2   volume          1324002 non-null  int64  
 3   transType       1324002 non-null  int64  
 4   transStatus     1324002 non-null  int64  
 5   sender_id   

In [ ]:
# Convert report_month to datetime type
payment_enriched['report_month'] = pd.to_datetime(payment_enriched['report_month'], format = '%Y-%m')

# Convert product_id & source_id to string for iteration
payment_enriched['product_id'] = payment_enriched['product_id'].astype(str)
payment_enriched['source_id'] = payment_enriched['source_id'].astype(str)

In [ ]:
# Checking Missing values
print(payment_enriched.isnull().sum())
print(transactions.isnull().sum())

report_month     0
payment_group    0
product_id       0
source_id        0
volume           0
category         0
team_own         0
dtype: int64
transaction_id          0
merchant_id             0
volume                  0
transType               0
transStatus             0
sender_id           49059
receiver_id        164795
extra_info        1317907
timeStamp               0
dtype: int64


In [ ]:
# Checking duplicates
print(transactions.duplicated())
print(payment_enriched.duplicated())

0          False
1          False
2          False
3          False
4          False
           ...  
1323997    False
1323998    False
1323999    False
1324000    False
1324001    False
Length: 1324002, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
892    False
893    False
894    False
895    False
896    False
Length: 897, dtype: bool


In [ ]:
# Check values
print(transactions.describe())

       transaction_id   merchant_id        volume     transType   transStatus  \
count    1.324002e+06  1.324002e+06  1.324002e+06  1.324002e+06  1.324002e+06   
mean     3.002233e+09  2.460318e+03  2.388059e+05  6.979222e+00 -1.204625e+01   
std      1.042606e+07  3.304277e+03  9.681009e+05  7.459714e+00  5.577823e+01   
min      3.000000e+09  5.000000e+00  1.000000e+00  2.000000e+00 -1.333000e+03   
25%      3.001121e+09  3.050000e+02  1.000000e+04  2.000000e+00  1.000000e+00   
50%      3.002200e+09  2.250000e+03  3.000000e+04  2.000000e+00  1.000000e+00   
75%      3.003255e+09  2.270000e+03  1.000000e+05  8.000000e+00  1.000000e+00   
max      6.000066e+09  1.625250e+05  7.869148e+07  3.000000e+01  1.000000e+00   

          sender_id   receiver_id     timeStamp  
count  1.274943e+06  1.159207e+06  1.324002e+06  
mean   1.033938e+08  2.084884e+08  1.683130e+12  
std    6.234305e+08  9.287794e+08  1.707815e+08  
min    1.000001e+07 -6.300000e+01  1.682874e+12  
25%    1.005657e+07 

Part II: Data Wrangling
Using payment_report.csv & product.csv
1. Top 3 product_ids with the highest volume.
2. Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?
3. Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.
4. Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?

5. Using transactions.csv
Define type of transactions (‘transaction_type’) for each row, given:
- transType = 2 & merchant_id = 1205: Bank Transfer Transaction
- transType = 2 & merchant_id = 2260: Withdraw Money Transaction
- transType = 2 & merchant_id = 2270: Top Up Money Transaction
- transType = 2 & others merchant_id: Payment Transaction
- transType = 8, merchant_id = 2250: Transfer Money Transaction
- transType = 8 & others merchant_id: Split Bill Transaction
- Remained cases are invalid transactions
6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.

In [ ]:
# 1. Top 3 product_ids with the highest volume

# group volume by product_id
df_top3 = payment_report.groupby('product_id')['volume'].sum()

# sort value decending
df_top3 = df_top3.sort_values(ascending=False)

# print top 3
print(df_top3.head(3))


product_id
1976    61797583647
429     14667676567
372     13713658515
Name: volume, dtype: int64


In [ ]:
# 2. Given that 1 product_id is only owed by 1 team, are there any abnormal products against this rule?

# group team by product_id
df_product_team = payment_enriched.groupby('product_id', as_index=False)['team_own'].nunique()

# abnormal products
print(df_product_team[df_product_team['team_own'] != 1])





Empty DataFrame
Columns: [product_id, team_own]
Index: []


In [ ]:
# 3. Find the team has had the lowest performance (lowest volume) since Q2.2023. Find the category that contributes the least to that team.

# filter value since 2023-04
df_team_volume = payment_enriched[(payment_enriched['report_month'] >= '2023-04-01')]

# group volume by team_own
df_team_volume = df_team_volume.groupby('team_own')['volume'].sum()

# sorting value
df_team_volume = df_team_volume.sort_values(ascending=True)

print(df_team_volume.head(1))

# filter volume & category by team APS and since 2023-04
df_aps = payment_enriched[(payment_enriched['team_own'] == 'APS') & (payment_enriched['report_month'] >= '2023-04-01')]

# group volume by category
df_aps = df_aps.groupby('category')['volume'].sum()

# sorting value
df_aps = df_aps.sort_values(ascending=True)

# find the category with lowest volume
print(df_aps.head(1))





team_own
APS    51141753
Name: volume, dtype: int64
category
PXXXXXE    25232438
Name: volume, dtype: int64


In [ ]:
# 4. Find the contribution of source_ids of refund transactions (payment_group = ‘refund’), what is the source_id with the highest contribution?

# filter value for refund transactions
df_refund = payment_report[payment_report['payment_group'] == 'refund']

# group source_id by sum
df_refund = df_refund.groupby('source_id')['volume'].sum()

# sort value decending
df_refund = df_refund.sort_values(ascending=False)

print(df_refund.head(1))



source_id
38    36527454759
Name: volume, dtype: int64


In [ ]:
# 5. Using transactions.csv Define type of transactions (‘transaction_type’) for each row

# df transaction_type
def transaction_type(row):
  if row['transType'] == 2 and row['merchant_id'] == 1205:
    return 'Bank Transfer Transaction'
  elif row['transType'] == 2 and row['merchant_id'] == 2260:
    return 'Withdraw Money Transaction'
  elif row['transType'] == 2 and row['merchant_id'] == 2270:
    return 'Top Up Money Transaction'
  elif row['transType'] == 2:
    return 'Payment Transaction'
  elif row['transType'] == 8 and row['merchant_id'] == 2250:
    return 'Transfer Money Transaction'
  elif row['transType'] == 8:
    return 'Split Bill Transaction'
  else:
    return 'Invalid'
transactions['transaction_type'] = transactions.apply(transaction_type, axis=1)

print(transactions[['transType', 'merchant_id' ,'transaction_type']])




         transType  merchant_id            transaction_type
0               24            5                     Invalid
1                2          305         Payment Transaction
2               22         7255                     Invalid
3                2         2270    Top Up Money Transaction
4                2         2275         Payment Transaction
...            ...          ...                         ...
1323997          2          305         Payment Transaction
1323998          2         2270    Top Up Money Transaction
1323999          2         2270    Top Up Money Transaction
1324000         22            5                     Invalid
1324001          8         2250  Transfer Money Transaction

[1324002 rows x 3 columns]


In [ ]:
# 6. Of each transaction type (excluding invalid transactions): find the number of transactions, volume, senders and receivers.
# filter data to exclude invalid transaction type
transactions = transactions[transactions['transaction_type'] != 'Invalid']

transactions = transactions.groupby('transaction_type').agg({'transaction_id': 'count', 'volume': 'sum', 'sender_id': 'nunique', 'receiver_id': 'nunique'})
print(transactions)

                            transaction_id        volume  sender_id  \
transaction_type                                                      
Bank Transfer Transaction            37879   50605806190      23156   
Payment Transaction                 398677   71851515181     139583   
Split Bill Transaction                1376       4901464       1323   
Top Up Money Transaction            290502  108606478829     110409   
Transfer Money Transaction          341177   37033171492      39021   
Withdraw Money Transaction           33725   23418181420      24814   

                            receiver_id  
transaction_type                         
Bank Transfer Transaction          9271  
Payment Transaction              113298  
Split Bill Transaction              572  
Top Up Money Transaction         110409  
Transfer Money Transaction        34585  
Withdraw Money Transaction        24814  
